In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
NAFCUSTOMER_C360_ACCOUNTS = dataiku.Dataset("NAFCUSTOMER_C360_ACCOUNTS")
NAFCUSTOMER_C360_ACCOUNTS_df = NAFCUSTOMER_C360_ACCOUNTS.get_dataframe()

ACCOUNT_BUNDLER_LIST = dataiku.Dataset("ACCOUNT_BUNDLER_LIST")
ACCOUNT_BUNDLER_LIST_df = ACCOUNT_BUNDLER_LIST.get_dataframe()

ACCOUNTS_PARTY_EXTRACT = dataiku.Dataset("Account_Party_extract")
ACCOUNTS_PARTY_EXTRACT_df = ACCOUNTS_PARTY_EXTRACT.get_dataframe()


/data/dataiku/dss_data/code-envs/python/env_clc/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [2]:
ACCOUNT_BUNDLER_LIST_df['IS_BUNDLER'] = True
ACCOUNT_BUNDLER_LIST_df = ACCOUNT_BUNDLER_LIST_df[['EDW_CUSTOMER_NAME','IS_BUNDLER']]

df = pd.merge(NAFCUSTOMER_C360_ACCOUNTS_df,ACCOUNT_BUNDLER_LIST_df, how='left', on='EDW_CUSTOMER_NAME')
df['DUNS'] = df['DUNS'].astype('Int64', errors='ignore')

df.loc[df["IS_BUNDLER"].isnull(),'IS_BUNDLER'] = False
df.loc[df["IS_BUNDLER"],'EDW_CUSTOMER_NAME'] = np.nan

# override --> need to figure out how to deal with these additions
df.loc[df["EDW_CUSTOMER_NAME"]=='EXXONMOBIL PL CONVERSION L1','IS_BUNDLER'] = True

In [14]:
# MDM matches, shared by Wes Corbin during the week of Nov 17, 2022
# key columns: ACCOUNTNUMBER, WEXBUSINESSID, NAME, DUNS

ACCOUNTS_PARTY_EXTRACT = dataiku.Dataset("Account_Party_extract")
ACCOUNTS_PARTY_EXTRACT_df = ACCOUNTS_PARTY_EXTRACT.get_dataframe()

print(len(ACCOUNTS_PARTY_EXTRACT_df))

ACCOUNTS_PARTY_EXTRACT_df = ACCOUNTS_PARTY_EXTRACT_df[~ACCOUNTS_PARTY_EXTRACT_df.ACCOUNTNUMBER.str.contains('-', na=False)]
ACCOUNTS_PARTY_EXTRACT_df['ACCOUNTNUMBER'] = ACCOUNTS_PARTY_EXTRACT_df['ACCOUNTNUMBER'].str.strip()
ACCOUNTS_PARTY_EXTRACT_df['ACCOUNTNUMBER'] = ACCOUNTS_PARTY_EXTRACT_df['ACCOUNTNUMBER'].astype('float')
ACCOUNTS_PARTY_EXTRACT_df['ACCOUNTNUMBER'] = ACCOUNTS_PARTY_EXTRACT_df['ACCOUNTNUMBER'].astype('Int64')

2149826


In [15]:
ACCOUNTS_PARTY_EXTRACT_df.dtypes

WEXBUSINESSID            int64
ACCOUNTPARTYROLE        object
EBXMATCHGROUPID        float64
GROUPSIZE                int64
DUNS                   float64
NAME                    object
ACCOUNTNUMBER            Int64
ACCOUNTPLATFORM_        object
RPLACCOUNTSETUPDATE     object
dtype: object

In [21]:
print(len(df))
ACCOUNTS_PARTY_EXTRACT_df.dropna(subset=['ACCOUNTNUMBER'], inplace=True)
df_j = pd.merge(df, ACCOUNTS_PARTY_EXTRACT_df, left_on='CUSTOMER_ACCOUNT_ID', right_on='ACCOUNTNUMBER', how='left')
print(len(df_j))

898003
898007


In [22]:
df_j.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,DUNS_x,IS_BUNDLER,WEXBUSINESSID,ACCOUNTPARTYROLE,EBXMATCHGROUPID,GROUPSIZE,DUNS_y,NAME,ACCOUNTNUMBER,ACCOUNTPLATFORM_,RPLACCOUNTSETUPDATE
0,9100008895186,METRO FOUNDATIONS,NaN,57123175,True,10923060.0,Customer,1.698569e+15,1.0,57123175.0,METRO FOUNDATIONS,9100008895186,SIEBEL,2021-03-10
1,9100003606921,CYRILLA LANDSCAPING,CYRILLA LANDSCAPING,146530154,False,10378993.0,Customer,1.698569e+15,1.0,146530154.0,Cyrilla Landscaping,9100003606921,SIEBEL,2015-11-17
2,6122004189767,GERMAIN PLUMB HEATING INC,GERMAIN PLUMB HEATING INC,161346598,False,9803397.0,Customer,1.698569e+15,1.0,161346598.0,Germain Plumb & Heating Inc,6122004189767,SIEBEL,2000-08-08
3,9100006987447,HOSLER MECHANICAL INC,NaN,121515209,True,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
4,9100009447507,MUNIZ EXECUTIVE ENTERPRISES,NaN,81875142,True,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN


In [0]:
ACCOUNTS_WITH_BUNDLER_df = df

# Write recipe outputs
ACCOUNTS_WITH_BUNDLER = dataiku.Dataset("ACCOUNTS_WITH_BUNDLER")
ACCOUNTS_WITH_BUNDLER.write_with_schema(ACCOUNTS_WITH_BUNDLER_df)